# Finetune Llama-3 with LLaMA Factory

Please use a **free** Tesla T4 Colab GPU to run this!

Project homepage: https://github.com/hiyouga/LLaMA-Factory

## Install Dependencies

In [ ]:
%cd /content/
%rm -rf LLaMA-Factory
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git
%cd LLaMA-Factory
%ls
!pip install -e .[torch,bitsandbytes,unsloth]

/content
Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 338, done.
remote: Counting objects: 100% (338/338), done.
remote: Compressing objects: 100% (269/269), done.
remote: Total 338 (delta 74), reused 206 (delta 54), pack-reused 0 (from 0)
Receiving objects: 100% (338/338), 9.59 MiB | 12.04 MiB/s, done.
Resolving deltas: 100% (74/74), done.
/content/LLaMA-Factory
assets/       docker/      LICENSE      pyproject.toml  requirements.txt  src/
CITATION.cff  evaluation/  Makefile     README.md       scripts/          tests/
data/         examples/    MANIFEST.in  README_zh.md    setup.py
Obtaining file:///content/LLaMA-Factory
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
try:
  assert torch.cuda.is_available() is True
except AssertionError:
  print("Please set up a GPU before using LLaMA Factory: https://medium.com/mlearning-ai/training-yolov4-on-google-colab-316f8fff99c6")

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ML_training_data/zlatoust_sentence_aligned_with_CATS.csv')

In [ ]:
df = df.loc[df['cos_sim'] >= 0.33]

In [ ]:
df['instruction'] = "Напиши упрощенную версию текста на русском языке."

In [ ]:
train_df, extra = train_test_split(df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(extra, test_size=0.5, random_state=42)

In [ ]:
def df_to_llamafactory_json(df):
  dicts = []
  for n, row in df.iterrows():
    d = {}
    d["instruction"] = row['instruction']
    d["input"] = row['source']
    d["output"] = row['target']
    dicts.append(d)
  return dicts

In [ ]:
train_data = df_to_llamafactory_json(train_df)
test_data = df_to_llamafactory_json(val_df)
val_data = df_to_llamafactory_json(test_df)

In [ ]:
test_data[:5]

[{'instruction': 'Напиши упрощенную версию текста на русском языке.',
  'input': 'Фандорину только двадцать один? ',
  'output': 'Ему всего двадцать один'},
 {'instruction': 'Напиши упрощенную версию текста на русском языке.',
  'input': 'Маша встает, берет подушку и уходит, сердитая.',
  'output': 'Маша встаёт, берёт подушку и уходит, сердитая.'},
 {'instruction': 'Напиши упрощенную версию текста на русском языке.',
  'input': '— спросил он',
  'output': '— спросил Сергей. — Почему кровь? '},
 {'instruction': 'Напиши упрощенную версию текста на русском языке.',
  'input': 'В таких мыслях она провела без него пять дней, те самые, которые он должен был находиться в отсутствии.',
  'output': 'В таких мыслях она провела без него пять дней. '},
 {'instruction': 'Напиши упрощенную версию текста на русском языке.',
  'input': 'Очень, очень оригинально, но… меня, собственно, не эта часть вашей статейки заинтересовала, а некоторая мысль, пропущенная в конце статьи, но которую вы, к сожалению, 

In [ ]:
import json

with open('/content/LLaMA-Factory/data/zlatoust_train.json', 'w') as f:
  json.dump(train_data, f, ensure_ascii = False)

with open('/content/LLaMA-Factory/data/zlatoust_val.json', 'w') as f:
  json.dump(val_data, f, ensure_ascii = False)

with open('/content/LLaMA-Factory/data/zlatoust_test.json', 'w') as f:
  json.dump(test_data, f, ensure_ascii = False)

In [ ]:
with open('/content/LLaMA-Factory/data/dataset_info.json', 'r') as f:
  dataset_info = json.load(f)

In [ ]:
dataset_info["zlatoust_train"] = {"file_name": "zlatoust_train.json"}
dataset_info["zlatoust_val"] = {"file_name": "zlatoust_val.json"}
dataset_info["zlatoust_test"] = {"file_name": "zlatoust_test.json"}

In [ ]:
with open('/content/LLaMA-Factory/data/dataset_info.json', 'w') as f:
  json.dump(dataset_info, f)

## Fine-tune model via LLaMA Board

In [ ]:
%cd /content/LLaMA-Factory/
!GRADIO_SHARE=1 llamafactory-cli webui

/content/LLaMA-Factory
2025-03-03 17:42:23.526586: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741023743.759709    2020 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741023743.820355    2020 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-03 17:42:24.378588: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
* Running on local URL:  http://0.0.0.0:7860
* Running on public URL: https://2838d4ff7401a0d0

## Fine-tune model via Command Line

It takes ~30min for training.

In [ ]:
import json

args = dict(
  stage="sft",                                               # do supervised fine-tuning
  do_train=True,
  model_name_or_path="unsloth/llama-3-8b-Instruct-bnb-4bit", # use bnb-4bit-quantized Llama-3-8B-Instruct model
  dataset="identity,alpaca_en_demo",                         # use alpaca and identity datasets
  template="llama3",                                         # use llama3 prompt template
  finetuning_type="lora",                                    # use LoRA adapters to save memory
  lora_target="all",                                         # attach LoRA adapters to all linear layers
  output_dir="llama3_lora",                                  # the path to save LoRA adapters
  per_device_train_batch_size=2,                             # the micro batch size
  gradient_accumulation_steps=4,                             # the gradient accumulation steps
  lr_scheduler_type="cosine",                                # use cosine learning rate scheduler
  logging_steps=5,                                           # log every 5 steps
  warmup_ratio=0.1,                                          # use warmup scheduler
  save_steps=1000,                                           # save checkpoint every 1000 steps
  learning_rate=5e-5,                                        # the learning rate
  num_train_epochs=3.0,                                      # the epochs of training
  max_samples=500,                                           # use 500 examples in each dataset
  max_grad_norm=1.0,                                         # clip gradient norm to 1.0
  loraplus_lr_ratio=16.0,                                    # use LoRA+ algorithm with lambda=16.0
  fp16=True,                                                 # use float16 mixed precision training
  report_to="none",                                          # disable wandb logging
)

json.dump(args, open("train_llama3.json", "w", encoding="utf-8"), indent=2)

%cd /content/LLaMA-Factory/

!llamafactory-cli train train_llama3.json

## Infer the fine-tuned model

In [ ]:
from llamafactory.chat import ChatModel
from llamafactory.extras.misc import torch_gc

%cd /content/LLaMA-Factory/

args = dict(
  model_name_or_path="unsloth/llama-3-8b-Instruct-bnb-4bit", # use bnb-4bit-quantized Llama-3-8B-Instruct model
  adapter_name_or_path="llama3_lora",                        # load the saved LoRA adapters
  template="llama3",                                         # same to the one in training
  finetuning_type="lora",                                    # same to the one in training
)
chat_model = ChatModel(args)

messages = []
print("Welcome to the CLI application, use `clear` to remove the history, use `exit` to exit the application.")
while True:
  query = input("\nUser: ")
  if query.strip() == "exit":
    break
  if query.strip() == "clear":
    messages = []
    torch_gc()
    print("History has been removed.")
    continue

  messages.append({"role": "user", "content": query})
  print("Assistant: ", end="", flush=True)

  response = ""
  for new_text in chat_model.stream_chat(messages):
    print(new_text, end="", flush=True)
    response += new_text
  print()
  messages.append({"role": "assistant", "content": response})

torch_gc()

## Merge the LoRA adapter and optionally upload model

NOTE: the Colab free version has merely 12GB RAM, where merging LoRA of a 8B model needs at least 18GB RAM, thus you **cannot** perform it in the free version.

In [ ]:
!huggingface-cli login

In [ ]:
import json

args = dict(
  model_name_or_path="meta-llama/Meta-Llama-3-8B-Instruct", # use official non-quantized Llama-3-8B-Instruct model
  adapter_name_or_path="llama3_lora",                       # load the saved LoRA adapters
  template="llama3",                                        # same to the one in training
  finetuning_type="lora",                                   # same to the one in training
  export_dir="llama3_lora_merged",                          # the path to save the merged model
  export_size=2,                                            # the file shard size (in GB) of the merged model
  export_device="cpu",                                      # the device used in export, can be chosen from `cpu` and `auto`
  # export_hub_model_id="your_id/your_model",               # the Hugging Face hub ID to upload model
)

json.dump(args, open("merge_llama3.json", "w", encoding="utf-8"), indent=2)

%cd /content/LLaMA-Factory/

!llamafactory-cli export merge_llama3.json